In [ ]:
import pandas as pd
import numpy as np
from pytorch_forecasting import TimeSeriesDataSet


contentDataframe = pd.read_excel('Data/Online Retail.xlsx')
contentDataframe.dropna(subset=['CustomerID','InvoiceDate'])
contentDataframe = contentDataframe.sort_values(by=["CustomerID"],kind="mergesort") #stable sorting means I can sort by one column instead of two :)


c:\Users\Jones\Desktop\CBAProj\venv\Lib\site-packages\pytorch_forecasting\models\base\_base_model.py:28: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
display(contentDataframe)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
61619,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.04,12346.0,United Kingdom
61624,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,-74215,2011-01-18 10:17:00,1.04,12346.0,United Kingdom
14938,537626,85116,BLACK CANDELABRA T-LIGHT HOLDER,12,2010-12-07 14:57:00,2.10,12347.0,Iceland
14939,537626,22375,AIRLINE BAG VINTAGE JET SET BROWN,4,2010-12-07 14:57:00,4.25,12347.0,Iceland
14940,537626,71477,COLOUR GLASS. STAR T-LIGHT HOLDER,12,2010-12-07 14:57:00,3.25,12347.0,Iceland
...,...,...,...,...,...,...,...,...
541536,581498,85099B,JUMBO BAG RED RETROSPOT,5,2011-12-09 10:26:00,4.13,NaN,United Kingdom
541537,581498,85099C,JUMBO BAG BAROQUE BLACK WHITE,4,2011-12-09 10:26:00,4.13,NaN,United Kingdom
541538,581498,85150,LADIES & GENTLEMEN METAL SIGN,1,2011-12-09 10:26:00,4.96,NaN,United Kingdom
541539,581498,85174,S/4 CACTI CANDLES,1,2011-12-09 10:26:00,10.79,NaN,United Kingdom


In [ ]:
contentDataframe.dropna(subset=['CustomerID'])
#Predict next purchase, so we want timestep obv where each transaction is one time step, can make daily rev for context 
#sort by date time so need to convert
contentDataframe['dateTime'] = pd.to_datetime(contentDataframe['InvoiceDate'])
contentDataframe.sort_values(by='dateTime', kind='mergesort', ascending=False)
contentDataframe['time_idx'] = contentDataframe.groupby(['CustomerID','dateTime']).cumcount()
contentDataframe['totalTxValue'] = contentDataframe['Quantity'] * contentDataframe['UnitPrice']

In [ ]:
validationRows = []
testingRows = []
for CustomerID, group in contentDataframe.groupby('CustomerID'):
    numberOfRows = len(group)
    last20 = int(np.floor(0.2*numberOfRows))
    last20Rows = group.tail(last20)
    validationSplit = group.head(int(last20 / 2))
    testingSplit = group.tail(int(last20Rows - validationSplit))
    validationRows.append(validationSplit)
    testingRows.append(testingSplit)
validationDataframe = pd.concat(validationRows, ignore_index=True)
testingDataframe = pd.concat(testingRows,ignore_index=True)

In [ ]:
#training = TimeSeriesDataSet
#instatiate data loaders

TypeError: TimeSeriesDataSet.__init__() missing 4 required positional arguments: 'data', 'time_idx', 'target', and 'group_ids'